In [94]:
import pandas as pd

In [196]:
#Version 1.0.3

def gen_formatter(input_df, percent=True, dollar=True, summary_mode=False):
    
    df = input_df.copy()
    d = dollar
    p = percent
    s_m = summary_mode
    
    formattable_types = ['float64', 'int64', 'int32']
    
    for column in df:
        if df[column].dtype not in formattable_types:
            pass
        elif df[column].dtype in formattable_types:
            if p and ('%' in column or 'percent' in column.lower()):
                if s_m:
                    df[column] = df[column].map('%{:.0f}'.format)
                else:
                    df[column] = df[column].map('%{}'.format)
            if d and ('$' in column or 'budget' in column.lower() or 'price' in column.lower()):
                df[column] = df[column].map('${:.2f}'.format)
            if df[column].dtype == 'float64':
                df[column] = df[column].map('{:,.1f}'.format)
            if df[column].dtype == 'int64':
                df[column] = df[column].map('{:,}'.format)
            
    return df


In [96]:
# creating a variable for the path to the csv
schools_data_path = 'Resources\schools_complete.csv'
students_data_path = 'Resources\students_complete.csv'

# use the path variable as the path in .read_csv() function
schools_data = pd.read_csv(schools_data_path)
students_data = pd.read_csv(students_data_path)

# oops convert to DataFrames
schools_df = pd.DataFrame(schools_data)
students_df = pd.DataFrame(students_data)

In [97]:
#create a function to find percentage
def calc_percent(subset, population):
    return round(100 * subset / population, 2)


In [98]:
schools_df.dtypes

School ID       int64
school_name    object
type           object
size            int64
budget          int64
dtype: object

In [99]:
# list of prefixes and suffixes found in cleaning_data.ipynb
pref_suff = ['Dr. ', 'Mr. ', 'Mrs. ', 'Ms. ', 'Miss ', ' MD', ' DDS', ' DVM', ' PhD']

# iterate through students_df['student_name'] column and replace each title with ''
for title in pref_suff:
    students_df['student_name'] = students_df['student_name'].str.replace(title, '', regex=True)

students_df.head(10)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84
5,5,Bryan Miranda,M,9th,Huang High School,94,94
6,6,Sheena Carter,F,11th,Huang High School,82,80
7,7,Nicole Baker,F,12th,Huang High School,96,69
8,8,Michael Roth,M,10th,Huang High School,95,87
9,9,Matthew Greene,M,10th,Huang High School,96,84


In [100]:
# merging both dataframes on column: 'school_name' // on='school_name' can also work here i think bc the names are ==
merged_df = pd.merge(students_df, schools_df, on=['school_name', 'school_name'])
merged_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [101]:
# using .count() as a DF method to show the number of rows in column x
student_total = merged_df['Student ID'].count()

# using the len() of Series.unique() -which returns a list of unique values- to count school total
# another way to do it would be: schools_df['school_name'].count()
school_total = len(merged_df['school_name'].unique())

print(f'students: {student_total}\nschools: {school_total}')


students: 39170
schools: 15


In [102]:
# total budget using Series.sum()
budget_total = schools_df['budget'].sum()
budget_total

24649428

In [103]:
# average math and reading scores
math_avg = merged_df['math_score'].mean()
reading_avg = merged_df['reading_score'].mean()
print(f'math average: {math_avg:.1f}\nreading average: {reading_avg:.1f}')

math average: 79.0
reading average: 81.9


In [104]:
# percent students who passed math, reading, overall -ppass stands for percent pass
passing_math = merged_df['math_score'] >= 70
passing_reading = merged_df['reading_score'] >= 70

# my way: uses .value_counts().get() where normalize=True
# math_ppass = passing_math.value_counts(normalize=True).get(True) * 100
# reading_ppass = passing_reading.value_counts(normalize=True).get(True) * 100

# module way:
passing_math_df = merged_df[passing_math]
passing_reading_df = merged_df[passing_reading]

# counting the rows in series: passing_math_df['student_name'] that was filtered ^
passing_math_count = passing_math_df['student_name'].count()
passing_reading_count = passing_reading_df['student_name'].count()
print(f'passing math: {passing_math_count:,}\npassing reading: {passing_reading_count:,}')

# math to find percents
math_ppass = passing_math_count * 100 / float(student_total)
reading_ppass = passing_reading_count * 100 / float(student_total)
print(f'\n% passing math: {math_ppass:.1f}%\n% passing reading: {reading_ppass:.1f}%')

# find passing both steps: 1) dataframe filter both criteria in [], 2) .count() the df on some column, 3) do math & print
passing_both_df = merged_df[passing_math & passing_reading]
passing_both_count = passing_both_df['Student ID'].count()
both_ppass = passing_both_count * 100 / float(student_total)


print(f'\n% passing both: {both_ppass:.1f}%')

passing math: 29,370
passing reading: 33,610

% passing math: 75.0%
% passing reading: 85.8%

% passing both: 65.2%


In [105]:
# testing my calc_percent() function
calc_percent(passing_math_count, student_total)

74.98

In [106]:
# creating the summary df
district_summary_df = pd.DataFrame([
    {'Total Schools': school_total,
     'Total Students': student_total,
     'Total Budget': budget_total,
     'Avg Math Score': math_avg,
     'Avg Reading Score': reading_avg,
     '% Passing Math': math_ppass,
     '% Passing Reading': reading_ppass,
     '% Passing Both': both_ppass}
])

district_summary_df

,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Both
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [107]:
# map() formatting
def format_summary():
    district_summary_df['Total Budget'] = district_summary_df['Total Budget'].map('${:,.2f}'.format)
    district_summary_df['Avg Math Score'] = district_summary_df['Avg Math Score'].map('{:.1f}'.format)
    district_summary_df['Avg Reading Score'] = district_summary_df['Avg Reading Score'].map('{:.1f}'.format)
    district_summary_df['% Passing Math'] = district_summary_df['% Passing Math'].map('{:.0f}'.format)
    district_summary_df['% Passing Reading'] = district_summary_df['% Passing Reading'].map('{:.0f}'.format)
    district_summary_df['% Passing Both'] = district_summary_df['% Passing Both'].map('{:.0f}'.format)
    district_summary_df['Total Students'] = district_summary_df['Total Students'].map('{:,}'.format)

# optional run format bc if its already been run, the values will be strings
# format_summary()
district_summary_df

,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Passing Both
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [108]:
# determine school types with the school name as index
per_school_types = schools_df.set_index(['school_name'])['type']
per_school_types

# add to a dataframe
school_index_df = pd.DataFrame(schools_df.set_index(['school_name']))
school_index_df

,School ID,type,size,budget
school_name,,,,
Huang High School,0,District,2917,1910635
Figueroa High School,1,District,2949,1884411
Shelton High School,2,Charter,1761,1056600
Hernandez High School,3,District,4635,3022020
Griffin High School,4,Charter,1468,917500
Wilson High School,5,Charter,2283,1319574
Cabrera High School,6,Charter,1858,1081356
Bailey High School,7,District,4976,3124928
Holden High School,8,Charter,427,248087


In [109]:
# using the df.set_index() from^ take series from different columns and calculations with those serieses
stu_per_school = school_index_df['size']
budget_per_school = school_index_df['budget']
budget_per_student = budget_per_school / stu_per_school
budget_per_student

school_name
Huang High School        655.0
Figueroa High School     639.0
Shelton High School      600.0
Hernandez High School    652.0
Griffin High School      625.0
Wilson High School       578.0
Cabrera High School      582.0
Bailey High School       628.0
Holden High School       581.0
Pena High School         609.0
Wright High School       583.0
Rodriguez High School    637.0
Johnson High School      650.0
Ford High School         644.0
Thomas High School       638.0
dtype: float64

In [110]:
# average math/reading using groupby() to isolate rows by school and return a df
student_index_df = students_df.groupby(['school_name'])

# these lines of code create variables to hold the groupby().mean() df
avg_math = student_index_df['math_score'].mean()
avg_reading = student_index_df['reading_score'].mean()

avg_reading

school_name
Bailey High School       81.033963
Cabrera High School      83.975780
Figueroa High School     81.158020
Ford High School         80.746258
Griffin High School      83.816757
Hernandez High School    80.934412
Holden High School       83.814988
Huang High School        81.182722
Johnson High School      80.966394
Pena High School         84.044699
Rodriguez High School    80.744686
Shelton High School      83.725724
Thomas High School       83.848930
Wilson High School       83.989488
Wright High School       83.955000
Name: reading_score, dtype: float64

In [111]:
 # To get the passing percentages, we need to:
 # 1. Determine what is the passing grade
 # 2. Get the number of students who passed math and reading
 # 3. Get the students who passed math and passed reading

# same code as before when we found count of students passing
#  only diff is .groupby(['school_name'])['math_score'] which creates a series of school and pass math counts
# we used the same variables here to reference the df made by: merged_df[merged_df['math_score'] >= 70]
# ^that is all in variable: passing_math_df
per_school_passing_math = passing_math_df.groupby(['school_name'])['math_score'].count()
per_school_passing_reading = passing_reading_df.groupby(['school_name'])['reading_score'].count()

# added the passing_both and specified the value column as ['Student ID']
per_school_passing_both = passing_both_df.groupby(['school_name']).count()['Student ID']
per_school_passing_both

school_name
Bailey High School       2719
Cabrera High School      1697
Figueroa High School     1569
Ford High School         1487
Griffin High School      1330
Hernandez High School    2481
Holden High School        381
Huang High School        1561
Johnson High School      2549
Pena High School          871
Rodriguez High School    2119
Shelton High School      1583
Thomas High School       1487
Wilson High School       2068
Wright High School       1626
Name: Student ID, dtype: int64

In [112]:
# finding percentages of passing math/reading but we already have the series which can have math applied to them
school_ppass_math = per_school_passing_math * 100 / stu_per_school
school_ppass_reading = per_school_passing_reading * 100 / stu_per_school
school_ppass_both = per_school_passing_both * 100 / stu_per_school

school_ppass_both

school_name
Bailey High School       54.642283
Cabrera High School      91.334769
Figueroa High School     53.204476
Ford High School         54.289887
Griffin High School      90.599455
Hernandez High School    53.527508
Holden High School       89.227166
Huang High School        53.513884
Johnson High School      53.539172
Pena High School         90.540541
Rodriguez High School    52.988247
Shelton High School      89.892107
Thomas High School       90.948012
Wilson High School       90.582567
Wright High School       90.333333
dtype: float64

In [113]:
# creating the per school summary
per_school_summary_df = pd.DataFrame({
    'School Type': per_school_types,
    'Total Students': stu_per_school,
    'Total School Budget': budget_per_school,
    'Per Student Budget': budget_per_student,
    'Average Math Score': avg_math,
    'Average Reading Score': avg_reading,
    '% Passing Math': school_ppass_math,
    '% Passing Reading': school_ppass_reading,
    '% Passing Both': school_ppass_both
})

def format_summary(df):
    df['Total Students'] = df['Total Students'].map('{:,}'.format)
    df['Total School Budget'] = df['Total School Budget'].map('${:,}'.format)
    df['Per Student Budget'] = df['Per Student Budget'].map('${:,.0f}'.format)
    df['Average Math Score'] = df['Average Math Score'].map('{:.1f}'.format)
    df['Average Reading Score'] = df['Average Reading Score'].map('{:.1f}'.format)
    df['% Passing Math'] = df['% Passing Math'].map('%{:.1f}'.format)
    df['% Passing Reading'] = df['% Passing Reading'].map('%{:.1f}'.format)
    df['% Passing Both'] = df['% Passing Both'].map('%{:.1f}'.format)


# format_summary(per_school_cummary_df)
per_school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both
school_name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


In [114]:
# sorting the values in overall passing
# had to comment out the formatting from the last cell to have floats and redo it after the sort
top_schools = per_school_summary_df.sort_values(['% Passing Both'], ascending=False)
format_summary(top_schools)
top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.1,84.0,%94.1,%97.0,%91.3
Thomas High School,Charter,"1,635","$1,043,130",$638,83.4,83.8,%93.3,%97.3,%90.9
Griffin High School,Charter,"1,468","$917,500",$625,83.4,83.8,%93.4,%97.1,%90.6
Wilson High School,Charter,"2,283","$1,319,574",$578,83.3,84.0,%93.9,%96.5,%90.6
Pena High School,Charter,962,"$585,858",$609,83.8,84.0,%94.6,%95.9,%90.5


In [115]:
# sorting bottom 5 schools
bottom_schools = per_school_summary_df.sort_values(['% Passing Both'], ascending=True)
format_summary(bottom_schools)
bottom_schools.head()


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363",$637,76.8,80.7,%66.4,%80.2,%53.0
Figueroa High School,District,"2,949","$1,884,411",$639,76.7,81.2,%66.0,%80.7,%53.2
Huang High School,District,"2,917","$1,910,635",$655,76.6,81.2,%65.7,%81.3,%53.5
Hernandez High School,District,"4,635","$3,022,020",$652,77.3,80.9,%66.8,%80.9,%53.5
Johnson High School,District,"4,761","$3,094,650",$650,77.1,81.0,%66.1,%81.2,%53.5


In [122]:
# grade level stats
# setting var to the merged_df filtered on the grade value
grade_nine = merged_df[merged_df['grade'] == '9th']
grade_ten = merged_df[merged_df['grade'] == '10th']
grade_eleven = merged_df[merged_df['grade'] == '11th']
grade_twelve = merged_df[merged_df['grade'] == '12th']

# math avg series creation
# creating a series with groupby().mean()
math_series_ninth = grade_nine.groupby(['school_name']).mean()['math_score']
math_series_tenth = grade_ten.groupby(['school_name']).mean()['math_score']
math_series_eleventh = grade_eleven.groupby(['school_name']).mean()['math_score']
math_series_twelfth = grade_twelve.groupby(['school_name']).mean()['math_score']

# reading avg series creation
reading_series_ninth = grade_nine.groupby(['school_name']).mean()['reading_score']
reading_series_tenth = grade_ten.groupby(['school_name']).mean()['reading_score']
reading_series_eleventh = grade_eleven.groupby(['school_name']).mean()['reading_score']
reading_series_twelfth = grade_twelve.groupby(['school_name']).mean()['reading_score']

# putting the values into dataframe
# math
grade_math_avg = pd.DataFrame({
    '9th': math_series_ninth,
    '10th': math_series_tenth,
    '11th': math_series_eleventh,
    '12th': math_series_twelfth
})

# reading
grade_reading_avg = pd.DataFrame({
    '9th': reading_series_ninth,
    '10th': reading_series_tenth,
    '11th': reading_series_eleventh,
    '12th': reading_series_twelfth
})

# formatting for per school avg --since replaced with gen_formatter()
def format_avg_df(df):
    for i in df:
        df[i] = df[i].map('{:.1f}'.format)
    df.index.name = None

gen_formatter(grade_math_avg)
grade_math_avg.head(4)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2


In [130]:
# get the spending ranges
# pd.cut() takes a 1d object and fits its components into bins then outputs a 1d object
spending_bins = [0, 585, 630, 645, 675]
budget_per_student.groupby(pd.cut(budget_per_student, spending_bins)).count()

(0, 585]      4
(585, 630]    4
(630, 645]    4
(645, 675]    3
dtype: int64

In [136]:
# naming the bins
group_names = ['<$585', '$585-629', '$630-644', '$645-675']

# categorize spending
# setting a new column in school summary = the cut() 1d object based on th spending bins
per_school_summary_df['Spending Ranges (Per Student)'] = pd.cut(budget_per_student, spending_bins, labels=group_names)
per_school_summary_df.head(3)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both,Spending Rnages (Per Student),Spending Ranges (Per Student)
school_name,,,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283,$585-629,$585-629
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769,<$585,<$585
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476,$630-644,$630-644


In [201]:
# stats per bin
# create 1D array to build the new df columns
math_score_bin = per_school_summary_df.groupby(['Spending Ranges (Per Student)']).mean()['Average Math Score']
reading_score_bin = per_school_summary_df.groupby(['Spending Ranges (Per Student)']).mean()['Average Reading Score']
math_passing_bin = per_school_summary_df.groupby(['Spending Ranges (Per Student)']).mean()['% Passing Math']
reading_passing_bin = per_school_summary_df.groupby(['Spending Ranges (Per Student)']).mean()['% Passing Reading']
overall_passing_bin = (math_passing_bin + reading_passing_bin)/ 2
# both_passing_bin = per_school_summary_df.groupby(['Spending Ranges (Per Student)']).mean()['% Passing Both']


# new dataframe with bins as columns
spending_summary_df = pd.DataFrame({
    'Average Math Score': math_score_bin,
    'Average Reading Score': reading_score_bin,
    '% Passing Math': math_passing_bin,
    '% Passing Reading': reading_passing_bin,
    '% Overall Passing': overall_passing_bin
})

# using the gen_formatter() on a copy of the df so it doesnt convert all the values to strings

gen_formatter(spending_summary_df, summary_mode=True)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.5,83.9,%93,%97,%95
$585-629,81.9,83.2,%87,%93,%90
$630-644,78.5,81.6,%73,%84,%79
$645-675,77.0,81.0,%66,%81,%74


In [202]:
# creating bins based on the school size
#stu_per_school.describe()
#  count      15.000000
#  mean     2611.333333
#  std      1420.915282 *
#  min       427.000000
#  25%      1698.000000
#  50%      2283.000000
#  75%      3474.000000
#  max      4976.000000

# making the bin ranges + naming them
school_size_bins = [0, 1000, 2000, 5000]
size_bin_names = ['<1000', '1000-2000', '2000-5000']

# creating a new column in the summary df to hold the bin info
per_school_cut = pd.cut(per_school_summary_df['Total Students'], school_size_bins, labels=size_bin_names)
per_school_summary_df['School Size Ranges'] = per_school_cut

# create the groupby() arrays that will be the columns
# grouping by the ranges for the bins we average each parameter to create each series
s_math_score_bin = per_school_summary_df.groupby(['School Size Ranges']).mean()['Average Math Score']
s_reading_score_bin = per_school_summary_df.groupby(['School Size Ranges']).mean()['Average Reading Score']
s_math_pass_bin = per_school_summary_df.groupby(['School Size Ranges']).mean()['% Passing Math']
s_reading_pass_bin = per_school_summary_df.groupby(['School Size Ranges']).mean()['% Passing Reading']
s_overall_pass_bin = (s_math_pass_bin + s_reading_pass_bin) / 2

# creating the df
school_size_summary_df = pd.DataFrame({
    'Average Math Score': s_math_score_bin,
    'Average Reading Score': s_reading_score_bin,
    '% Passing Math': s_math_pass_bin,
    '% Passing Reading': s_reading_pass_bin,
    '% Overall Passing': s_overall_pass_bin
})

# school_size_summary_df
gen_formatter(school_size_summary_df, summary_mode=True)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size Ranges,,,,,
<1000,83.8,83.9,%94,%96,%95
1000-2000,83.4,83.9,%94,%97,%95
2000-5000,77.7,81.3,%70,%83,%76


In [219]:
# summary by school type

# calc the values by school type
t_math_score_bin = per_school_summary_df.groupby(['School Type']).mean()['Average Math Score']
t_reading_score_bin = per_school_summary_df.groupby(['School Type']).mean()['Average Reading Score']
t_math_pass_bin = per_school_summary_df.groupby(['School Type']).mean()['% Passing Math']
t_reading_pass_bin = per_school_summary_df.groupby(['School Type']).mean()['% Passing Reading']
t_overall_pass_bin = per_school_summary_df.groupby(['School Type']).mean()['% Passing Both']

# form the dataframe
school_type_summary = pd.DataFrame({
    'Average Math Score': t_math_score_bin,
    'Average Reading Score': t_reading_score_bin,
    '% Passing Math': t_math_pass_bin,
    '% Passing Reading': t_reading_pass_bin,
    '% Overall Passing': t_overall_pass_bin
    
})

gen_formatter(school_type_summary, summary_mode=True)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.5,83.9,%94,%97,%90
District,77.0,81.0,%67,%81,%54
